# Convolutional Neural Networks (CNNs)

### Welcome to the 11th Lab of 42028: Deep Learning and CNN!

In this  Lab/Tutorial session you will be implementing simple ResNet50 module and create a Convolutional Neural Network for image classification .

So lets get started!

## Tutorial:
Implementation of Simple ResNet50 based Micro-ResNet CNN architecture using Keras for classfication of FashionMNIST dataset.

## Tasks for this week:

1. Implementation of Simple ResNet50 based Mini-ResNet CNN architecture for FashionMNIST image classification using Keras API.
2. Train and test model


**Reference and adapted from:** https://github.com/alinarw/ResNet/blob/master/ResNet.ipynb 

## The ResNet Begins !

### Step 1: Import required packages

we will need tensorflow, numpy, os and keras


In [0]:
import keras
from keras.layers.core import Layer
import keras.backend as K
import tensorflow as tf

from keras.models import Model
from keras.layers import Conv2D, MaxPool2D,  \
    Dropout, Dense, Input, concatenate,      \
    GlobalAveragePooling2D, AveragePooling2D,\
    Flatten, BatchNormalization, Activation, Add

from keras.layers import MaxPooling2D, Input
from keras.initializers import glorot_uniform

import cv2 
import numpy as np 
from keras.datasets import cifar10, fashion_mnist 
from keras import backend as K 
from keras.utils import np_utils

from keras import backend as K
from keras.regularizers import l2

import math 
from keras.optimizers import SGD, Adam, Adadelta 
from keras.callbacks import LearningRateScheduler
from keras.activations import relu, softmax
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from sklearn.metrics import confusion_matrix

from keras.utils import plot_model
from keras.utils.vis_utils import model_to_dot
from IPython.display import HTML, display, clear_output, SVG

import warnings
warnings.filterwarnings("ignore")


### Step-2: Design the Simple ResNet-50 module

<img src='http://drive.google.com/uc?export=view&id=1spP_NXWTUqJG-Mlw9VKC3jxpgCEriYCa' alt='Conv'> 




In [0]:
# Residual block conv(1X1) -> BN -> relu -> conv(3x3) -> bn -> relu -> conv(1x1) ->BN

def res_block(x, filters):

    print('Input Shape x.shape', x.shape)
    # Create 1X1 CONV, --> BatchNormalization --> Activation, Initialize weights with Xavier uniform initialization (glorot_uniform)
    conv1 = Conv2D(filters=filters, kernel_size=(1, 1), strides=(2, 2), padding='same', 
                   kernel_initializer=glorot_uniform(seed=0))(x)
    bn1 = BatchNormalization()(conv1)
    act1 = Activation('relu')(bn1)
    print('conv1.shape', conv1.shape)

    # Create 3X3 CONV, --> BatchNormalization --> Activation, Initialize weights with Xavier uniform initialization (glorot_uniform)
    conv2 = Conv2D(filters=filters, kernel_size=(3, 3), strides=(1, 1), padding='same', 
                   kernel_initializer=glorot_uniform(seed=0))(act1)
    bn2 = BatchNormalization()(conv2)
    act2 = Activation('relu')(bn2)
    print('conv2.shape', conv2.shape)

    # Create 1X1 CONV, --> BatchNormalization, Initialize weights with Xavier uniform initialization (glorot_uniform)
    conv3 = Conv2D(filters=filters, kernel_size=(1, 1), strides=(1, 1), padding='same', 
                   kernel_initializer=glorot_uniform(seed=0))(act2)
    bn3 = BatchNormalization()(conv3)
    print('conv3.shape', conv3.shape)

    # Create 1X1 CONV on the Input to re-shape
    x = Conv2D(filters=filters, kernel_size=(1, 1), strides=(2, 2), padding='same')(x)

    out = Add()([bn3, x])
    
    return out

In [0]:
kernel_init = keras.initializers.glorot_uniform()
bias_init = keras.initializers.Constant(value=0.2)

### Step 3: Design a Micro-ResNet with 1 Layer of Residual Block

<img src='http://drive.google.com/uc?export=view&id=1PITOPO_fNskYWDS6GEIGKJn-FItNuw_j' alt='Conv'>





In [0]:
# Define Images dimension and number of classes
img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 1)
num_classes = 10

# Create the input layer
input_layer = Input(shape=(img_rows, img_cols, 1)) # Use the actual input size

# Create 7X7X64 CONV --> BatchNormalization --> Activation 
x = Conv2D(64, padding='same', 
           kernel_initializer='he_normal', 
           kernel_size=7, strides=2,
           )(input_layer)
x = BatchNormalization()(x)
x = Activation('relu')(x)

# Add MaxPOOL, 3X3, Pad: Same, Stride: 2
x = MaxPooling2D(pool_size=3,padding='same', strides=2)(x)

# Add 1 ResNet block, 64 filter
res_block1 = res_block(x, 64)
print('---------block 1 end-----------')

# Create Classifier Block
# Add Flatten layer
classifer_Block = Flatten()(res_block1)

# Add Dense Layer, 1000 nodes, Activation - ReLU
classifer_Block = Dense(1000,
                activation='relu')(classifer_Block)

# Add Dense Layer, 10 nodes, Activation = SoftMax
outputs = Dense(num_classes,
                activation='softmax')(classifer_Block)

In [0]:
# Form the model
model = Model(input_layer, outputs, name='Micro_ResNet')

In [0]:
#Display Model Summary
model.summary()

plot_model(model, to_file='model.png', show_layer_names=True, show_shapes=True, rankdir='TB')
SVG(model_to_dot(model, show_layer_names=True, show_shapes=True, rankdir='TB').create(prog='dot', format='svg'))

### Step 4: Download the Fashion-MNIST dataset using keras

In [0]:
num_classes = 10 #Number of classes in the dataset

def load_fashion_mnist_data(img_rows, img_cols):

    # Load cifar10 training and validation sets
    (X_train, Y_train), (X_valid, Y_valid) = fashion_mnist.load_data()
    
    # Resize training images
    X_train = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_train[:,:,:]])
    X_valid = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_valid[:,:,:]])

    # Transform targets to keras compatible format
    Y_train = np_utils.to_categorical(Y_train, num_classes)
    Y_valid = np_utils.to_categorical(Y_valid, num_classes)
    
    X_train = X_train.astype('float32')
    X_valid = X_valid.astype('float32')

    # preprocess data
    X_train = X_train / 255.0
    X_valid = X_valid / 255.0

    return X_train, Y_train, X_valid, Y_valid

In [0]:
# Display the shapes of the training images
X_train, y_train, X_test, y_test = load_fashion_mnist_data(28, 28)
X_train = X_train.reshape(X_train.shape[0],28,28,1)
X_test = X_test.reshape(X_test.shape[0],28,28,1)
print(X_train.shape)
print(X_test.shape)

In [0]:
## Display an image from the dataset
import matplotlib.pyplot as plt
plt.imshow(X_train.reshape(X_train.shape[0], 28, 28)[100])

### Step 5: Define the Hyper-parameters, Compile the model, Start training

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:
#Start training the model
history1 = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=15, batch_size=100)


### Step 6: Display Training and Validation accuracy curve

In [0]:
## Plot the Traning and Validation loss

loss = history1.history['accuracy']
val_loss = history1.history['val_accuracy']

epochs = range(len(loss))

plt.figure()

plt.plot(epochs, loss, 'r', label='Training accuracy')
plt.plot(epochs, val_loss, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

In [0]:
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

## Your task begins now!

## Task 1: Design a Mini-ResNet 

### Add three ResNet50 layers in the Mico-ResNet architecture created above in the following order and configuration:

<img src='http://drive.google.com/uc?export=view&id=1nhCprmNy2MqGM2W4De_pP2-L2tk0K5uU' alt='Conv'>



In [0]:
## WRITE YOUR CODE HERE ## 
# Define Images dimension and number of classes
img_rows, img_cols = #Add the image rows and cols
input_shape =  # Define the Input shape
num_classes = # Define the number of Classes

# Create the input layer


# Create 7X7X64 CONV --> BatchNormalization --> Activation 
x =   # Add CONV Layer
x =   # Add Batch Normalization
x =   # Add Activation

# Add MaxPOOL, 3X3, Pad: Same, Stride: 2
x =   # Add MaxPool

# Add 3 ResNet block, 64 filter, 256 filters, and 512 filters
res_block1 = # Add ResNet Block
print('---------block 1 end-----------')
res_block2 = # Add ResNet Block
print('---------block 2 end-----------')
res_block3 = # Add ResNet Block
print('---------block 3 end-----------')

# Create Classifier Block
# Add Flatten layer
classifer_Block = 

# Add Dense Layer, 1000 nodes, Activation - ReLU
classifer_Block =  # Add Dense Layer with ReLU and 1000 Nodes

# Add Dense Layer, 10 nodes, Activation = SoftMax
outputs =  # Add Dense Layer with SoftMax and 10 Nodes

## END YOUR CODE ##

In [0]:
## WRITE YOUR CODE HERE ##
# Create and compile the Model
model = 

# END YOUR CODE ##

In [0]:
## WRITE YOUR CODE HERE ##
# Print Model Summary


# Create the Graph image and display


# END YOUR CODE ##

## Task 2: Define the HyperParamenters, Optimzer, etc and compile model

In [0]:
## WRITE YOUR CODE HERE ##
# Compile the model


# END YOUR CODE ##

## Task 3: Train the Model

In [0]:
## WRITE YOUR CODE HERE ##
#Start training the model, USE: EPOCH=15, BATCH_SIZE=100
history2 = 

# END YOUR CODE ##

## Task 4: Test on Train and Test set

In [0]:
## WRITE YOUR CODE HERE ##
# Test the model on Test and Validationa dataset



# END YOUR CODE ##

## Task 5: Display the Train and Validation Accuracy curve

In [0]:
## Plot the Traning and Validation loss

loss = history2.history['accuracy']
val_loss = history2.history['val_accuracy']

epochs = range(len(loss))

plt.figure()

plt.plot(epochs, loss, 'r', label='Training accuracy')
plt.plot(epochs, val_loss, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()